In [6]:
import hashlib
import json
import random
from time import time
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import rsa, padding

class Blockchain:
    def __init__(self):
        self.chain = []
        self.candidate_votes = {'candidate_A': 0, 'candidate_B': 0}

    def add_block(self, votes, previous_hash=None):
        block = {
            'index': len(self.chain) + 1,
            'timestamp': time(),
            'votes': votes,
            'previous_hash': previous_hash or self.hash(self.chain[-1]) if self.chain else None,
        }

        # Update the candidate votes count
        self.update_candidate_votes(votes)

        self.chain.append(block)
        return block

    def add_vote(self, voter_id, candidate, private_key):
        new_vote = {
            'voter_id': voter_id,
            'candidate': candidate,
        }

        # Validate the vote using the private key
        if self.validate_vote(new_vote, private_key):
            # Create a new block with the current vote
            new_block = self.add_block(votes=[new_vote], previous_hash=self.hash(self.chain[-1]) if self.chain else None)
            return new_block

    def validate_vote(self, vote, private_key):
        try:
            # Sign the vote with the private key
            signature = private_key.sign(
                json.dumps(vote, sort_keys=True).encode(),
                padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
                hashes.SHA256()
            )

            # Verify the signature
            public_key = private_key.public_key()
            public_key.verify(
                signature,
                json.dumps(vote, sort_keys=True).encode(),
                padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
                hashes.SHA256()
            )

            return True

        except Exception as e:
            print(f"Vote validation failed: {e}")
            return False

    def update_candidate_votes(self, votes):
        for vote in votes:
            candidate = vote['candidate']
            self.candidate_votes[candidate] += 1

    def count_votes(self):
        total_votes = 0
        for block in self.chain:
            total_votes += len(block['votes'])
        return total_votes

    def get_candidate_votes(self):
        return self.candidate_votes

    @staticmethod
    def hash(block):
        # Hash a block using SHA-256
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    @property
    def last_block(self):
        return self.chain[-1] if self.chain else None

# Test Case 1: No Tampering
def test_no_tampering():
    print("\nTest Case 1 - No Tampering")

    # Generate private and public keys for each voter
    voter_private_keys = [rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend()) for _ in range(15)]
    voter_ids = [f'user{i}' for i in range(1, 16)]

    # Create a blockchain
    blockchain = Blockchain()

    # Add votes to the blockchain, each creating a new block
    for voter_id, private_key in zip(voter_ids, voter_private_keys):
        candidate = random.choice(['candidate_A', 'candidate_B'])  # Each voter randomly votes for a candidate
        blockchain.add_vote(voter_id=voter_id, candidate=candidate, private_key=private_key)

    # Count total votes
    total_votes = blockchain.count_votes()
    print(f"Total Votes: {total_votes}")

    # Get candidate votes
    candidate_votes = blockchain.get_candidate_votes()
    print("Candidate Votes:")
    for candidate, votes in candidate_votes.items():
        print(f"{candidate}: {votes} votes")

# Run Test Case 1
test_no_tampering()
class TamperingDetectedException(Exception):
    pass

# Test Case 2: Tampering in Between
def test_tampering():
    print("\nTest Case 2 - Tampering in Between")

    # Generate private and public keys for each voter
    voter_private_keys = [rsa.generate_private_key(public_exponent=65537, key_size=2048, backend=default_backend()) for _ in range(15)]
    voter_ids = [f'user{i}' for i in range(1, 16)]

    # Create a blockchain
    blockchain = Blockchain()

    # Add votes to the blockchain, each creating a new block
    for i, (voter_id, private_key) in enumerate(zip(voter_ids, voter_private_keys)):
        candidate = random.choice(['candidate_A', 'candidate_B'])  # Each voter randomly votes for a candidate

        # Tampering in the middle (e.g., someone tries to use the private key of the 8th voter to cast a vote)
        if i != 7:  # Skip the 8th voter
            try:
                # Attempt to cast a vote using the private key of the 8th voter
                blockchain.add_vote(voter_id='user8', candidate=candidate, private_key=private_key)
                raise TamperingDetectedException("Tampering Detected! An individual tried to use the private key of the 8th voter.")
            except Exception as e:
                # Output the exception message (e.g., private key mismatch)
                print(f"Tampering Attempt Detected! {e}")
                raise TamperingDetectedException("Tampering Detected! An individual tried to use the private key of the 8th voter.")

        # Continue with normal voting process for other voters
        blockchain.add_vote(voter_id=voter_id, candidate=candidate, private_key=private_key)

    # Count total votes
    total_votes = blockchain.count_votes()
    print(f"Total Votes: {total_votes}")

    # Get candidate votes
    candidate_votes = blockchain.get_candidate_votes()
    print("Candidate Votes:")
    for candidate, votes in candidate_votes.items():
        print(f"{candidate}: {votes} votes")

# Run Test Case 2
try:
    test_tampering()
except TamperingDetectedException as e:
    print(f"\nProgram Terminated: {e}")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")






Test Case 1 - No Tampering
Total Votes: 15
Candidate Votes:
candidate_A: 8 votes
candidate_B: 7 votes

Test Case 2 - Tampering in Between
Tampering Attempt Detected! Tampering Detected! An individual tried to use the private key of the 8th voter.

Program Terminated: Tampering Detected! An individual tried to use the private key of the 8th voter.
